In [1]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import Neo4jVector
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI as Chat
import os

load_dotenv()

True

In [2]:
openai_api_key = os.getenv("OPENAI_API_KEY")
model = os.getenv("OPENAI_MODEL", "gpt-4o")
temperature = float(os.getenv("OPENAI_TEMPERATURE", 0))

NEO4J_URL = os.getenv("NEO4J_URL")
NEO4J_USERNAME = os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")

In [3]:
loader = TextLoader(
    "/Users/akileshjayakumar/Documents/neo4j-llm-fundamentals/rag_test_files/Climate_Change_and_Impact.txt")

documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()

In [4]:
llm = Chat(
    openai_api_key=openai_api_key,
    model=model,
    temperature=temperature
)

print(llm)

client=<openai.resources.chat.completions.Completions object at 0x117d797f0> async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x117d7b560> root_client=<openai.OpenAI object at 0x1177e0800> root_async_client=<openai.AsyncOpenAI object at 0x117d79850> model_name='gpt-4o' temperature=0.0 model_kwargs={} openai_api_key=SecretStr('**********')


In [5]:
db = Neo4jVector.from_documents(
    docs, OpenAIEmbeddings(), url=NEO4J_URL, username=NEO4J_USERNAME, password=NEO4J_PASSWORD
)

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL (row) { ... }} {position: line: 1, column: 21, offset: 20} for query: "UNWIND $data AS row CALL { WITH row MERGE (c:`Chunk` {id: row.id}) WITH c, row CALL db.create.setNodeVectorProperty(c, 'embedding', row.embedding) SET c.`text` = row.text SET c += row.metadata } IN TRANSACTIONS OF 1000 ROWS "


In [6]:
query = "Climate change refers to?"
docs_with_score = db.similarity_search_with_score(query, k=2)

In [7]:
for doc, score in docs_with_score:
    print("-" * 80)
    print("Score: ", score)
    print(doc.page_content)
    print("-" * 80)

--------------------------------------------------------------------------------
Score:  0.9485931396484375
Climate change refers to long-term shifts in temperatures and weather patterns, primarily caused by human activities such as burning fossil fuels. The impact of climate change includes rising sea levels, increased frequency of extreme weather events, and biodiversity loss. Addressing climate change requires global cooperation, and actions like reducing carbon emissions and promoting sustainable practices are key to mitigation.
--------------------------------------------------------------------------------


In [12]:
index_name = "vector"  # default index name

store = Neo4jVector.from_existing_index(
    embeddings,
    url=NEO4J_URL,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
    index_name=index_name,
)

print(store)

In [13]:
# First we create sample data in graph
store.query(
    "CREATE (p:Person {name: 'Tomaz', location:'Slovenia', hobby:'Bicycle', age: 33})"
)

[]

In [21]:
# Now we initialize from existing graph
existing_graph = Neo4jVector.from_existing_graph(
    embedding=OpenAIEmbeddings(),
    url=NEO4J_URL,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
    index_name="person_index",
    node_label="Person",
    text_node_properties=["name", "location"],
    embedding_node_property="embedding",
)
result = existing_graph.similarity_search("Slovenia", k=1)

In [22]:
result[0]

Document(metadata={'age': 33, 'hobby': 'Bicycle'}, page_content='\nname: Tomaz\nlocation: Slovenia')

In [23]:
retriever = store.as_retriever()
retriever.invoke(query)[0]

Document(metadata={'source': '/Users/akileshjayakumar/Documents/neo4j-llm-fundamentals/rag_test_files/Climate_Change_and_Impact.txt'}, page_content='Climate change refers to long-term shifts in temperatures and weather patterns, primarily caused by human activities such as burning fossil fuels. The impact of climate change includes rising sea levels, increased frequency of extreme weather events, and biodiversity loss. Addressing climate change requires global cooperation, and actions like reducing carbon emissions and promoting sustainable practices are key to mitigation.')

In [24]:
from langchain.chains import RetrievalQAWithSourcesChain

chain = RetrievalQAWithSourcesChain.from_chain_type(
    llm, chain_type="stuff", retriever=retriever
)

print(chain)

verbose=False combine_documents_chain=StuffDocumentsChain(verbose=False, llm_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['question', 'summaries'], input_types={}, partial_variables={}, template='Given the following extracted parts of a long document and a question, create a final answer with references ("SOURCES"). \nIf you don\'t know the answer, just say that you don\'t know. Don\'t try to make up an answer.\nALWAYS return a "SOURCES" part in your answer.\n\nQUESTION: Which state/country\'s law governs the interpretation of the contract?\n=========\nContent: This Agreement is governed by English law and the parties submit to the exclusive jurisdiction of the English courts in  relation to any dispute (contractual or non-contractual) concerning this Agreement save that either party may apply to any court for an  injunction or other relief to protect its Intellectual Property Rights.\nSource: 28-pl\nContent: No Waiver. Failure or delay in exercising any right o

In [26]:
chain.invoke(
    {"question": "Climate change refers to?"},
    return_only_outputs=True,
)

{'answer': 'Climate change refers to long-term shifts in temperatures and weather patterns, primarily caused by human activities such as burning fossil fuels. The impact of climate change includes rising sea levels, increased frequency of extreme weather events, and biodiversity loss. Addressing climate change requires global cooperation, and actions like reducing carbon emissions and promoting sustainable practices are key to mitigation.\n\n',
 'sources': '/Users/akileshjayakumar/Documents/neo4j-llm-fundamentals/rag_test_files/Climate_Change_and_Impact.txt'}